In [1]:
# import libraries
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

# read in data, taking 10% sample
p = .1
required_cols = ['JobID','Partition', 'Account', 'Group', 'GID',
       'User', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed',
       'ExitCode', 'State', 'NNodes', 'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES',
       'ReqTRES', 'Timelimit', 'NodeList', 'JobName']
accre = pd.read_csv('data/accre_data.txt', header=0, sep='|', usecols=required_cols, skiprows=lambda i: i>0 and random.random() > p)

## DISCOVERY

In [2]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251528 entries, 0 to 1251527
Data columns (total 22 columns):
JobID        1251528 non-null object
Partition    1240654 non-null object
Account      1251528 non-null object
Group        1251528 non-null object
GID          1251528 non-null int64
User         1251528 non-null object
Submit       1251528 non-null object
Eligible     1251528 non-null object
Start        1251528 non-null object
End          1251528 non-null object
Elapsed      1251528 non-null object
ExitCode     1251528 non-null object
State        1251528 non-null object
NNodes       1251528 non-null int64
NCPUS        1251528 non-null int64
ReqCPUS      1251528 non-null int64
ReqMem       1251528 non-null object
ReqGRES      8210 non-null object
ReqTRES      1240654 non-null object
Timelimit    1240654 non-null object
NodeList     1251528 non-null object
JobName      1251528 non-null object
dtypes: int64(4), object(18)
memory usage: 210.1+ MB


In [3]:
# look at the shape of the dataset
accre.shape

(1251528, 22)

## CLEANUP

In [4]:
# convert partition & account to category
accre.Partition = accre.Partition.astype('category')
accre.Account = accre.Account.astype('category')
accre.dtypes

JobID          object
Partition    category
Account      category
Group          object
GID             int64
User           object
Submit         object
Eligible       object
Start          object
End            object
Elapsed        object
ExitCode       object
State          object
NNodes          int64
NCPUS           int64
ReqCPUS         int64
ReqMem         object
ReqGRES        object
ReqTRES        object
Timelimit      object
NodeList       object
JobName        object
dtype: object

In [5]:
# convert Submit and Start to datetime
accre['Submit'] = pd.to_datetime(accre['Submit'], format='%Y/%m/%d')
accre['Start'] = pd.to_datetime(accre['Start'], format='%Y/%m/%d')

In [6]:
# create Queue_Time column = Start - Submit
accre['Queue_Time'] = accre['Start'] - accre['Submit']
accre.Queue_Time.head()

0   00:00:02
1   01:25:36
2   03:51:59
3   05:46:40
4   00:00:06
Name: Queue_Time, dtype: timedelta64[ns]

In [7]:
# check data types
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251528 entries, 0 to 1251527
Data columns (total 23 columns):
JobID         1251528 non-null object
Partition     1240654 non-null category
Account       1251528 non-null category
Group         1251528 non-null object
GID           1251528 non-null int64
User          1251528 non-null object
Submit        1251528 non-null datetime64[ns]
Eligible      1251528 non-null object
Start         1251528 non-null datetime64[ns]
End           1251528 non-null object
Elapsed       1251528 non-null object
ExitCode      1251528 non-null object
State         1251528 non-null object
NNodes        1251528 non-null int64
NCPUS         1251528 non-null int64
ReqCPUS       1251528 non-null int64
ReqMem        1251528 non-null object
ReqGRES       8210 non-null object
ReqTRES       1240654 non-null object
Timelimit     1240654 non-null object
NodeList      1251528 non-null object
JobName       1251528 non-null object
Queue_Time    1251528 non-null timedel

In [8]:
# head data exploration for each column
for column in list(accre.columns):
    print(accre[column].value_counts(normalize=False,ascending=False).head(10))
    print('')
    print('')
    

26828657_97     2
19224420_101    2
26827147_22     2
26827147_25     2
16891962_8      2
26763523_9      2
16866123        2
26827147_54     2
20958426        2
17368082        1
Name: JobID, dtype: int64


production    1231860
pascal           5602
maxwell          3100
debug              48
bigbox             25
mic                18
fermi               1
Name: Partition, dtype: int64


cms                  355141
beam_lab             145424
cms_lowprio           92562
p_csb_meiler          75941
cms_stage2            65972
mip_eecs              57702
h_biostat_student     44649
lola                  44270
p_meiler              42859
math                  37237
Name: Account, dtype: int64


cms                  507799
beam_lab             150418
p_csb_meiler          73480
mip_eecs              57820
p_meiler              45650
cms_stage1            45015
lola                  45006
h_biostat_student     44613
math                  37238
rokaslab              29939
Name: Group, dty

## BEGIN WORK ON QUESTION

In [9]:
#get count of status
state_messages = accre.State.value_counts().to_frame().reset_index()
state_messages.columns = ["status","count"]
state_messages.head(20)

import re
condition = re.compile("CANCELLED by\s\d*")
state_messages['status'] = state_messages.status.replace(to_replace=condition,value="CANCELLED")
state_messages.groupby("status")["count"].sum().to_frame().reset_index()


,status,count
0,CANCELLED,427862
1,COMPLETED,715374
2,FAILED,90685
3,NODE_FAIL,200
4,PREEMPTED,1391
5,TIMEOUT,16016


In [56]:
# get jobs run per node (count of JobID grouped by NodeList)
jobs_per_node = accre.JobID.groupby(accre['NodeList']).agg('count')
jobs_per_node = jobs_per_node.to_frame()
jobs_per_node = jobs_per_node.iloc[1:,:]
jobs_per_node = jobs_per_node.sort_values(by="JobID",ascending=False)
jobs_per_node.head()

,JobID
NodeList,
vmp1409,9395
vmp1242,8119
vmp1257,7546
vmp1001,6568
vmp1298,4794


In [58]:
#jobs failed per node (count of JobID grouped by NodeList from )
state_failed = ['NODE_FAIL']
state_bad = accre.loc[accre['State'].isin(state_failed)]
fails_per_node = state_bad.JobID.groupby(state_bad['NodeList']).agg('count')
fails_per_node = fails_per_node.to_frame()
fails_per_node = fails_per_node.sort_values(by="JobID",ascending=False)
fails_per_node.head()

,JobID
NodeList,
vmp214,3
vmp1423,3
vmp1335,3
vmp1352,3
vmp692,3


In [59]:
node_fails_table = pd.merge(jobs_per_node,fails_per_node,on='NodeList',how='left')
node_fails_table = node_fails_table.rename({'JobID_x':'JobRuns','JobID_y':'JobFails'},axis=1)
node_fails_table.head()

,JobRuns,JobFails
NodeList,,
vmp1409,9395,NaN
vmp1242,8119,NaN
vmp1257,7546,NaN
vmp1001,6568,1.0
vmp1298,4794,NaN


In [60]:
#fill NA of JobFails with 0
node_fails_table = node_fails_table.dropna()

In [61]:
node_fails_table['Fail%'] = node_fails_table.JobFails/node_fails_table.JobRuns

In [62]:
node_fails_table.sort_values(by='JobRuns', ascending=False)

,JobRuns,JobFails,Fail%
NodeList,,,
vmp1001,6568,1.0,0.000152
vmp1038,3024,1.0,0.000331
vmp1019,2802,1.0,0.000357
vmp667,2796,2.0,0.000715
vmp1032,2531,1.0,0.000395
vmp1331,2530,2.0,0.000791
vmp1035,2514,1.0,0.000398
vmp1023,2500,1.0,0.000400
vmp1062,2487,1.0,0.000402


In [67]:
condition1 = node_fails_table.JobFails != 0
condition2 = node_fails_table.JobRuns > 100

node_fails_table = node_fails_table[condition1&condition2]

In [69]:
node_fails_table = node_fails_table_new.sort_values(by='Fail%',ascending=False)
node_fails_table

,JobRuns,JobFails,Fail%
NodeList,,,
vmp692,101,3.0,0.029703
vmp695,111,2.0,0.018018
vmp598,101,1.0,0.009901
vmp698,110,1.0,0.009091
vmp594,114,1.0,0.008772
vmp425,561,3.0,0.005348
gpu0006,198,1.0,0.005051
vmp373,972,3.0,0.003086
vmp1423,998,3.0,0.003006
